In [2471]:
!pip install gensim


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2472]:
#import gensim.downloader as api
import gensim
from gensim.models import Word2Vec , KeyedVectors
#wv = api.load('word2vec-google-news-300')
wv = KeyedVectors.load_word2vec_format('/Users/bhaskaradhikari/nlp/GoogleNews-vectors-negative300.bin.gz', binary=True)



In [2722]:
import nltk
from gensim.utils import simple_preprocess

In [2723]:
import pandas as pd
import numpy as np

df = pd.read_csv('Messages.csv')
df_key = pd.read_csv('Keywords.csv')

In [2724]:
df

,message
0,1 desk
1,"flat pack desk, cupboard, pedestal drawers"
2,door step delivery
3,ground floor
4,No number i'm afraid - i can wait here on the web
5,1 day bed
6,"Yes, it is the frame for a single bed and 2 ma..."
7,"Ground floor to ground floor, it will be disma..."
8,"Just the bed frame, which will be dismantled a..."
9,Specific time


In [2725]:
number_of_duplicates = df.duplicated().sum()
number_of_duplicates

1333

In [2726]:
duplicate_rows = df[df.duplicated(keep=False)]
print("Duplicate rows:")
print(duplicate_rows)

Duplicate rows:
                                                message
0                                                1 desk
3                                          ground floor
9                                         Specific time
10                                           Yes please
14                                 first floor, no lift
24                                              no lift
25                                          first floor
28                                         Ground floor
29                                         Ground floor
36                                         Ground floor
37                                         Ground floor
63                                              No lift
65                                                   Ok
75                                      It’s a bungalow
77                                              No lift
78                                            Thank you
85                              

In [2727]:
df = df.drop_duplicates(keep=False)
df = df.reset_index(drop=True)

In [2732]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [2728]:
import re
corpus = []
nltk.download('stopwords')
from nltk.corpus import stopwords
for i in range(0, len(df)):
    review = re.sub('[^a-zA-Z]', ' ', df['message'][i])
    review = review.lower()
    review = review.split()
    
   # review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bhaskaradhikari/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2729]:
[[i,j,k] for i,j,k in zip(list(map(len,corpus)),corpus, df['message']) if i<1]

[[0, '', '44481'],
 [0, '', '43902'],
 [0, '', '43963'],
 [0, '', '890890'],
 [0, '', '234567'],
 [0, '', '44204'],
 [0, '', '44086'],
 [0, '', '44541'],
 [0, '', '44542'],
 [0, '', '👍'],
 [0, '', '7773.4'],
 [0, '', '5678'],
 [0, '', '44206'],
 [0, '', '44225'],
 [0, '', '15'],
 [0, '', '44549'],
 [0, '', '44203'],
 [0, '', '44480'],
 [0, '', '15530'],
 [0, '', '? ?'],
 [0, '', '44210'],
 [0, '', '44287'],
 [0, '', '44558'],
 [0, '', '44317'],
 [0, '', '44216'],
 [0, '', '44200'],
 [0, '', '8'],
 [0, '', '3'],
 [0, '', '1233'],
 [0, '', '44252'],
 [0, '', '7539'],
 [0, '', '44211'],
 [0, '', '44198'],
 [0, '', '44470'],
 [0, '', '44220'],
 [0, '', '44218'],
 [0, '', '???'],
 [0, '', '44245'],
 [0, '', '44224'],
 [0, '', '44532'],
 [0, '', '44197'],
 [0, '', '44233'],
 [0, '', '44230'],
 [0, '', '44379'],
 [0, '', '44231']]

In [2730]:
from nltk import sent_tokenize

words=[]
for sent in corpus:
    sent_token=sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))


In [2731]:
words

/Users/bhaskaradhikari/anaconda3/lib/python3.11/site-packages/IPython/core/displayhook.py:281: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


[['flat', 'pack', 'desk', 'cupboard', 'pedestal', 'drawers'],
 ['door', 'step', 'delivery'],
 ['no', 'number', 'afraid', 'can', 'wait', 'here', 'on', 'the', 'web'],
 ['day', 'bed'],
 ['yes',
  'it',
  'is',
  'the',
  'frame',
  'for',
  'single',
  'bed',
  'and',
  'mattresses'],
 ['ground',
  'floor',
  'to',
  'ground',
  'floor',
  'it',
  'will',
  'be',
  'dismantled'],
 ['just',
  'the',
  'bed',
  'frame',
  'which',
  'will',
  'be',
  'dismantled',
  'and',
  'the',
  'mattress'],
 ['ok', 'll', 'wait', 'for', 'your', 'confirmation', 'thanks'],
 ['business',
  'move',
  'large',
  'printers',
  'requiring',
  'four',
  'people',
  'to',
  'lift',
  'shelving',
  'storage',
  'units'],
 ['first', 'floor', 'no', 'lift', 'but', 'wide', 'staircase'],
 ['men'],
 ['can', 'you', 'quote', 'for', 'two', 'people', 'then'],
 ['can', 'just', 'confirm', 'what', 'that', 'includes', 'please'],
 ['ok',
  'sorry',
  'don',
  'think',
  'was',
  'clear',
  'it',
  'business',
  'move',
  'so',

In [2733]:
df_key

,keyword
0,basement
1,bungalow
2,delivery
3,downstair
4,front door
5,garage
6,garden
7,home
8,level
9,pavement


In [2734]:
corpus_key=[]
for i in range(len(df_key)):
    review= re.sub('[^a-zA-Z0-9]', ' ' ,df_key['keyword'][i])
    review=review.lower()
    review=review.split()
    review=' '.join(review)
    corpus_key.append(review)
corpus_key

['basement',
 'bungalow',
 'delivery',
 'downstair',
 'front door',
 'garage',
 'garden',
 'home',
 'level',
 'pavement',
 'pickup',
 'right now',
 'stairs',
 'storage',
 'street level']

In [2735]:
model=gensim.models.Word2Vec(words,window=3,min_count=2)

In [2736]:
model.wv.index_to_key


['to',
 'and',
 'the',
 'floor',
 'it',
 'no',
 'you',
 'is',
 'lift',
 'ground',
 'of',
 'sofa',
 'will',
 'bed',
 'for',
 'seater',
 'from',
 'just',
 'be',
 'house',
 'can',
 'on',
 'in',
 'with',
 'one',
 'up',
 'need',
 'there',
 'have',
 'table',
 'boxes',
 'thanks',
 'that',
 'small',
 'me',
 'double',
 'but',
 'first',
 'my',
 'at',
 'time',
 'would',
 'not',
 'flat',
 'get',
 'back',
 'tv',
 'please',
 'so',
 'do',
 'two',
 'pick',
 'quote',
 'stairs',
 'if',
 'we',
 'chair',
 'mattress',
 'yes',
 'ok',
 'chest',
 'sorry',
 'chairs',
 'are',
 'door',
 'drawers',
 'fridge',
 'bags',
 'wardrobe',
 'how',
 'its',
 'items',
 'much',
 'this',
 'as',
 'delivery',
 'thank',
 'know',
 'price',
 'st',
 'all',
 'about',
 'freezer',
 'your',
 'here',
 'large',
 'an',
 'rob',
 'only',
 'unit',
 'size',
 'van',
 'll',
 'hi',
 'medium',
 'specific',
 'both',
 'number',
 'what',
 'move',
 'sure',
 'flexible',
 'help',
 'dining',
 'some',
 'or',
 'single',
 'am',
 'machine',
 'think',
 'addre

In [2737]:
model.corpus_count

4298

In [2738]:
model.wv.similar_by_word('delivery')

[('at', 0.9988510608673096),
 ('flat', 0.9987675547599792),
 ('house', 0.9986815452575684),
 ('going', 0.9986386299133301),
 ('nd', 0.9985151886940002),
 ('drop', 0.9985073208808899),
 ('up', 0.9984943270683289),
 ('both', 0.998382031917572),
 ('but', 0.9982490539550781),
 ('on', 0.9982395172119141)]

In [2739]:
def find_sentences_with_similar_words(corpus, similar_words):
    """
    Search for sentences in the corpus that contain any of the similar words.

    Parameters:
    - corpus: List[str], the corpus of text (each element is a sentence or document).
    - similar_words: List[str], words similar to the search word.

    Returns:
    - Dict[str, List[str]], a dictionary where keys are similar words and values are lists of sentences containing those words.
    """
    # Initialize a dictionary to hold the results
    results = {word: [] for word in similar_words}
    
    # Iterate over the corpus to find sentences containing any of the similar words
    for sentence in corpus:
        for word in similar_words:
            if word in sentence.lower():  # Case-insensitive search
                results[word].append(sentence)
        
    counts = {word: len(sentences) for word, sentences in results.items()}
    
    data_for_df = [(word, "; ".join(sentences)) for word, sentences in results.items()]
 
    return results, counts , data_for_df


# Assuming 'model' is your trained Word2Vec model
#similar_words = [word for word, _ in model.wv.similar_by_word('delivery')]
similar_words=corpus_key
# Assuming 'corpus' is a list of sentences

# Find sentences containing similar words
sentences_with_similar_words , sentence_counts , data_for_df = find_sentences_with_similar_words(corpus, similar_words)

# Print the results
for word, sentences in sentences_with_similar_words.items():
    print(f"Sentences containing '{word}':")
    for sentence in sentences:
        print(f"- {sentence}")
    print()  # Blank line for readability

df_counts = pd.DataFrame(list(sentence_counts.items()), columns=['Word', 'Number of Sentences'])
df_sentences = pd.DataFrame(data_for_df, columns=['Word', 'Sentences'])



Sentences containing 'basement':
- no lift pick up first floor flat delivery is basement flat
- basement flat to basement flat no lift
- basement flat no lift so they have to take stairs one floor
- basement no stairs to climb
- destination is ground floor apartment moving from duplex ground floor and basement bedroom items from basement stairs
- other end is a basement
- basement no lift around steepish narrow stairs

Sentences containing 'bungalow':
- going to a bungalow
- my place gf i dont know about the lady with the carpet bungalow
- a bungalow
- bungalow property
- ground floor flat to bungalow and back
- ground floor flat to bungalow
- on ground floor one into a garage the other items into a ground floor bungalow and collect sofa from a house that has a few steps going up to it
- it s in first floor flat going in to a bungalow
- we live in a bungalow so no stairs my daughters garage is on the flat outside her house
- you don t need a lift as l have said we live in a bungalow ev

In [2740]:
df_counts_sorted = df_counts.sort_values(by='Number of Sentences', ascending=False)
df_counts_sorted

,Word,Number of Sentences
12,stairs,190
2,delivery,102
13,storage,50
6,garden,45
5,garage,41
3,downstair,31
4,front door,30
1,bungalow,26
7,home,22
8,level,21


In [2741]:
def get_sentences_by_word(df, word):
    """
    Retrieve all sentences containing a particular word from a DataFrame.

    Parameters:
    - df: DataFrame, the DataFrame containing words and their associated sentences.
    - word: str, the word for which to retrieve sentences.

    Returns:
    - str, sentences containing the word.
    """
    # Find the row in the DataFrame where 'Word' matches the specified word
    sentences = df.loc[df['Word'] == word, 'Sentences'].values
    
    # Check if any sentences were found
    if sentences.size > 0:
        return sentences[0]
    else:
        return "No sentences found for the word."

# Example usage
word_to_find = "basement"
found_sentences = get_sentences_by_word(df_sentences, word_to_find)

print(f"Sentences containing '{word_to_find}':")
print(found_sentences)


Sentences containing 'basement':
no lift pick up first floor flat delivery is basement flat; basement flat to basement flat no lift; basement flat no lift so they have to take stairs one floor; basement no stairs to climb; destination is ground floor apartment moving from duplex ground floor and basement bedroom items from basement stairs; other end is a basement; basement no lift around steepish narrow stairs


In [2742]:
from collections import Counter
import itertools

# Assuming 'words' is your list of lists of words
all_words = list(itertools.chain(*words))

# Count the words
word_counts = Counter(all_words)

# Get the list of words and their frequencies, ranked from most to least frequent
words_frequency_rank = word_counts.most_common()

# Display the ranked list
print("Words and their frequencies, ranked:")
for rank, (word, freq) in enumerate(words_frequency_rank, start=1):
    print(f"{rank}. {word}: {freq}")


Words and their frequencies, ranked:
1. to: 1218
2. and: 1047
3. the: 941
4. floor: 928
5. it: 650
6. no: 619
7. you: 544
8. is: 523
9. lift: 512
10. ground: 508
11. of: 413
12. sofa: 358
13. will: 352
14. bed: 349
15. for: 340
16. seater: 337
17. from: 330
18. just: 323
19. be: 320
20. house: 316
21. can: 305
22. on: 280
23. in: 280
24. with: 256
25. one: 250
26. up: 246
27. need: 242
28. there: 239
29. have: 231
30. table: 226
31. boxes: 219
32. thanks: 215
33. that: 203
34. small: 203
35. me: 197
36. double: 195
37. but: 192
38. first: 190
39. my: 189
40. at: 185
41. time: 182
42. would: 178
43. not: 172
44. flat: 171
45. get: 169
46. back: 164
47. tv: 164
48. please: 155
49. so: 154
50. do: 145
51. two: 144
52. pick: 136
53. quote: 135
54. stairs: 133
55. we: 132
56. if: 132
57. chair: 130
58. mattress: 129
59. yes: 128
60. ok: 127
61. chest: 126
62. sorry: 124
63. chairs: 124
64. are: 122
65. door: 120
66. drawers: 117
67. fridge: 113
68. bags: 112
69. how: 111
70. wardrobe: 111
7

In [2743]:
# Assuming corpus_key is your list of specific keywords
corpus_key_set = set(corpus_key)  # Convert to a set for efficient lookup

# Assuming words_frequency_rank is already calculated and contains tuples of (word, frequency)
# Filtered to only include words that are also in corpus_key
common_words_frequency = [(word, freq) for word, freq in words_frequency_rank if word in corpus_key_set]

# Display the common words and their frequencies
print("Common words and their frequencies, ranked:")
for rank, (word, freq) in enumerate(common_words_frequency, start=1):
    print(f"{rank}. {word}: {freq}")


Common words and their frequencies, ranked:
1. stairs: 133
2. delivery: 104
3. garden: 52
4. storage: 50
5. garage: 42
6. bungalow: 26
7. level: 25
8. home: 23
9. pickup: 17
10. basement: 9
11. pavement: 3


In [2744]:
def avg_word2vec(doc):
    # Assuming model.wv.vector_size is the embedding dimension
    valid_embeddings = [model.wv[word] for word in doc if word in model.wv.index_to_key]
    if valid_embeddings:
        return np.mean(valid_embeddings, axis=0)
    else:
        return np.zeros(model.wv.vector_size)  # Return a zero array if no valid embeddings


In [2746]:
!pip install tqdm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2745]:
from tqdm import tqdm

In [2747]:
#apply for the entire sentences
import numpy as np
X=[]
for i in tqdm(range(len(words))):
    X.append(avg_word2vec(words[i]))

  8%|███▏                                  | 364/4298 [00:00<00:01, 3636.77it/s]

100%|█████████████████████████████████████| 4298/4298 [00:00<00:00, 5940.72it/s]


In [2748]:
X

[array([-0.11099285,  0.24775712,  0.03089182,  0.01884277,  0.07731794,
        -0.39111334,  0.09289528,  0.48204747, -0.22796693, -0.10781527,
        -0.02413213, -0.35482058, -0.04066904,  0.13889623,  0.07629804,
        -0.18060248,  0.0849916 , -0.28296322, -0.06821091, -0.48911032,
         0.11124092,  0.00162131,  0.10839798, -0.07955068, -0.05521247,
         0.09179085, -0.07995166, -0.18842074, -0.20851156,  0.04010054,
         0.29292053, -0.01057553,  0.11816911, -0.23955922, -0.07302272,
         0.1981658 ,  0.07196094, -0.14287174, -0.08999231, -0.4122293 ,
         0.01220352, -0.21073034, -0.12191365,  0.02133382,  0.1437446 ,
        -0.10787763, -0.1271801 , -0.00994338,  0.27244288,  0.10080008,
         0.05403522, -0.12561515,  0.05630577, -0.03840639, -0.09348737,
         0.20744398,  0.10399409, -0.06692275, -0.24717966,  0.13671449,
         0.07979961,  0.03687572,  0.00782932, -0.04938769, -0.21175294,
         0.30287963,  0.14092486,  0.23580046, -0.3